## https://github.com/JuhyoenOh?tab=repositories

## 1 .Iris데이터에 대해서 5겹 교차검증(k-fold cross validation)을 사용하여 분류하시오.

In [96]:
from sklearn.model_selection import cross_val_score 
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
iris = load_iris()

In [97]:
X,y = iris.data, iris.target

In [117]:
rf = RandomForestClassifier(random_state = 2020)
score_list = cross_val_score(rf, X, y)

In [118]:
result  = list(map(lambda x : '{score:.2f}'.format(score=x), score_list))
print(result)

['0.97', '0.97', '0.90', '0.93', '1.00']


In [119]:
score_list  = cross_val_score(rf,X,y,cv=5)
score_list

array([0.96666667, 0.96666667, 0.9       , 0.93333333, 1.        ])

## 2.  Fashion MNIST Dataset 을 CNN으로 분류하시오

In [161]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import matplotlib.pyplot as plt
%matplotlib inline

In [162]:
seed =  2020

np.random.seed(seed)
tf.random.set_seed(seed)

In [163]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28,28,1).astype('float32')/255
X_test = X_test.reshape(X_test.shape[0], 28,28,1).astype('float32')/255
Y_train = tf.keras.utils.to_categorical(Y_train)
Y_test = tf.keras.utils.to_categorical(Y_test)

In [164]:
model = Sequential([
    Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation = 'relu'),
    Dropout(0.5),
    Dense(10, activation = 'softmax')
])
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 128)               1179776   
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)             

In [165]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [166]:
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

In [167]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=3,
                    batch_size=200, #epochs=5, verbose=2, 
                    callbacks=[early_stopping_callback, checkpointer])

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 48s 798us/step - loss: 0.2682 - accuracy: 0.9179 - val_loss: 0.0527 - val_accuracy: 0.9825

Epoch 00001: val_loss improved from inf to 0.05273, saving model to ./model/mnist-cnn-01-0.0527.hdf5
Epoch 2/3
60000/60000 [==============================] - 48s 798us/step - loss: 0.0900 - accuracy: 0.9733 - val_loss: 0.0389 - val_accuracy: 0.9866

Epoch 00002: val_loss improved from 0.05273 to 0.03887, saving model to ./model/mnist-cnn-02-0.0389.hdf5
Epoch 3/3
60000/60000 [==============================] - 47s 779us/step - loss: 0.0672 - accuracy: 0.9798 - val_loss: 0.0367 - val_accuracy: 0.9876

Epoch 00003: val_loss improved from 0.03887 to 0.03674, saving model to ./model/mnist-cnn-03-0.0367.hdf5


In [169]:
from tensorflow.keras.models import load_model
del model
model = load_model('model/mnist-cnn-03-0.0367.hdf5')

In [170]:
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test, verbose=0)[1]))


 Test Accuracy: 0.9876


## 3. IMDB영화 리류 데이터에 대하여 딥 러닝을 이용하여 감성분석을 하시오.

In [171]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tensorflow import keras
from keras.datasets import imdb

In [172]:
review_train = pd.read_csv('./dataset/labeledTrainData.tsv', header=0,sep="\t", quoting=3)
review_test = pd.read_csv('./dataset/testData.tsv', header=0,sep="\t", quoting=3)

In [173]:
import re

review_df['review'] = review_df['review'].str.replace('<br />', ' ' )
review_df['review'] = review_df['review'].apply(lambda x : re.sub("[^a-zA-Z]"," ",x))

In [174]:
review_df = review_train+review_test

In [175]:
review_df.head(3)

,id,review,sentiment
0,"""5814_8""""12311_10""","""With all this stuff going down at the moment ...",NaN
1,"""2381_9""""8348_2""","""\""The Classic War of the Worlds\"" by Timothy ...",NaN
2,"""7759_3""""5828_4""","""The film starts with a manager (Nicholas Bell...",NaN


In [176]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df.drop(['id', 'sentiment'], axis=1, inplace = False)

X_train, X_test, y_train, y_test= train_test_split(feature_df,class_df, test_size =0.2,
                                                  random_state=2020)

X_train.shape,X_test.shape

((20000, 1), (5000, 1))

In [184]:
from keras.preprocessing.sequence import pad_sequences
max_len=500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

ValueError: invalid literal for int() with base 10: 'review'

In [177]:
X_train

,review
13366,"""Well I'm probably about to be lambasted by ev..."
24884,"""Financially strapped Paramount pulled out all..."
1479,"""Maybe the greatest film ever about jazz.<br /..."
473,"""I got to see this just this last Friday at th..."
3487,"""With this film, Bunuel manipulates the viewer..."
...,...
18523,"""The New Batman Adventures (also called Gotham..."
11971,"""Nine out of ten might seem like a high mark t..."
14966,"""The barbarians maybe´s not the best film that..."
7491,"""When I saw the preview, I thought: this is go..."


In [178]:
X_test

,review
15753,"""well after watching this i can say that it ai..."
24385,"""My friends and I rented this for \""Bad Movie ..."
15641,"""Twin brothers separated at birth (Due to the ..."
19101,"""I watched the pilot and noticed more than a f..."
15684,"""Since I first saw Anchors Aweigh in 1945, vie..."
...,...
11178,"""Everything in this film is bad , the story , ..."
11416,"""Okay...so i've seen a lot of really odd/unusu..."
23514,"""This movie is the only movie to feature a sce..."
6780,"""A fabulous book about a fox and his family wh..."


In [179]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing.sequence import pad_sequences

model = Sequential()
model.add(Embedding(5000, 120))
model.add(LSTM(120))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 120)         600000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 120)               115680    
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 121       
Total params: 715,801
Trainable params: 715,801
Non-trainable params: 0
_________________________________________________________________


In [180]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [181]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('model/imdb_lstm_best_model.h5', monitor='val_accuracy', 
                     verbose=1, mode='max', save_best_only=True)

In [182]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10,
                    batch_size=64, callbacks=[mc, es])

C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 20000 samples, validate on 5000 samples
Epoch 1/10


ValueError: could not convert string to float: '"I saw this film on its release, and have watched it 3 or 4 more times, including last week. I regret I have to be a voice of dissension with regard to Mr. Branagh\'s performance.<br /><br />This is really a glorious, sumptuous film, to say nothing of ambitious at over 4 hours long - beautifully shot and designed. Derek Jacobi, Julie Christie, Kate Winslet, Richard Briers, and many others do fine jobs. Then there\'s Kenneth Branagh. If ever there was a vanity project for an actor, this is it, and Mr. Branagh spares nothing in putting the \\"ham\\" in Hamlet. From the stunt casting (which gives us the worst performance ever by the woefully miscast Jack Lemmon), to the bits of distracting business thrown in to infuse a sense of \\"naturalness,\\" to his own performance which runs the gamut from throwing away the single most famous soliloquy in all of literature to screaming every line of others. His performance confirms that, while he may c